In [1]:
import ee

In [7]:
# Trigger the authentication flow.
ee.Authenticate()
# 4/1AbUR2VNsudZwKwIf3Pd6nglRp7KHrD3seuudhf2IQrJut7GGm4DE59CiHi8


Successfully saved authorization token.


In [2]:
# Initialize the library.
ee.Initialize()

In [6]:
sentinel_1 = ee.ImageCollection("COPERNICUS/S1_GRD")
sentinel_2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [35]:
start_date = '2022-01-01'
end_date = '2023-06-01'
# lst_sentinel2 = sentinel_2.select('B4','B3','B2').filter(start_date,end_date)

In [3]:
import geopandas as gpd
from pyproj import CRS
baleh_file = gpd.read_file('D:\\OneDrive - The University of Nottingham\\Dissertation\\Baleh_project\\shp\\available_date.shp')
# special_data = baleh_file.iloc[368,369:1,5]
# target_crs = CRS.from_epsg(32648)
# shp_df = special_data.to_crs(target_crs)

In [4]:
baleh_file[baleh_file['Category']=="Mone"]

,Category,ID,Latitude,Longitude,Date,geometry
7713,Mone,58.0,20.4799,94.2553,2019-01-21 11:29:11 UTC,POINT (94.25530 20.47990)
7714,Mone,58.0,20.4799,94.2553,2019-01-28 08:42:51 UTC,POINT (94.25530 20.47990)
7715,Mone,58.0,20.4799,94.2553,2019-02-14 02:12:15 UTC,POINT (94.25530 20.47990)
7716,Mone,58.0,20.4799,94.2553,2019-05-15 23:47:42 UTC,POINT (94.25530 20.47990)
7717,Mone,58.0,20.4799,94.2553,2019-06-13 02:45:22 UTC,POINT (94.25530 20.47990)
...,...,...,...,...,...,...
7955,Mone,58.0,20.4799,94.2553,2023-01-16 11:18:11 UTC,POINT (94.25530 20.47990)
7956,Mone,58.0,20.4799,94.2553,2023-01-24 08:03:58 UTC,POINT (94.25530 20.47990)
7957,Mone,58.0,20.4799,94.2553,2023-01-28 06:26:43 UTC,POINT (94.25530 20.47990)
7958,Mone,58.0,20.4799,94.2553,2023-02-01 04:49:00 UTC,POINT (94.25530 20.47990)


Ambuclao 1; Angat 305; Bakun 663; Baluchaung-2 804(not use-low resolution); Bang_Lang 1012; Bersia 1381; Mone 7714;
Sempor 12127; Sesan 12454; Shwegyin_(KyaukNaga) 12674; Sirikit 12909; Sirindhorn 13128; Song_Hinh_1 13427; SungaiLayang 13720;
Temengor 14106

In [5]:
import ee
from datetime import datetime
from shapely.geometry import Polygon

row_368 = baleh_file.iloc[7714]
row_368_poi = ee.Geometry.Point(row_368[3],row_368[2])

# buffer_distance = 6000  # 定义一个缓冲区距离（以米为单位）
# roi = row_368_poi.buffer(buffer_distance)  # 在目标点周围创建一个缓冲区

cc = gpd.read_file('D:\\OneDrive - The University of Nottingham\\Dissertation\\Data\\example\\shp\\Mone.shp')
coordinates = cc.geometry.values[0].exterior.coords
polygon = Polygon(coordinates)
geometry = ee.Geometry.Polygon(list(polygon.exterior.coords))

#denfine Sentinel-2 image collection
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(row_368_poi) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 1)) \
    .filterDate('2019-12-01', '2019-12-30')  # time range

# define clip function
def clip_all(img):
    return img.clip(geometry)

# use clip function
s2_clip = s2.map(clip_all)

# obtain images list
s2_list = s2_clip.toList(s2_clip.size())

# 循环处理每个图像
for i in range(s2_list.size().getInfo()):
    img = ee.Image(s2_list.get(i))

    # 获取日期信息
    date = ee.Date(img.date()).format('yyyyMMdd').getInfo()

    # 添加图层
    vis_params = {'bands': ['B4', 'B3', 'B2'], 'max': 5048}
    # folium.Map.add_ee_layer(img, vis_params, 'Image_' + date)

    # 导出图像
    task = ee.batch.Export.image.toDrive(
        image= img.select(['B4', 'B3', 'B2']).visualize(**vis_params),
        description='Mone_' + date,
        region=geometry.getInfo()['coordinates'],
        maxPixels=1e10,
        fileNamePrefix='Mone_' + date,
        folder='Satellite Images',
        scale=5  # 替换为你需要的分辨率
    )
    task.start()

# 等待导出任务完成
while task.status()['state'] == 'RUNNING':
    pass

print('Please wait a moment')

Please wait a moment
